# SMART SCIENCE APPRENTICE BOT- TFIDF

Load Required Library

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import nltk
# nltk.download('stopwords')
!pip install underthesea

     |████████████████████████████████| 7.1MB 15.1MB/s 
     |████████████████████████████████| 757kB 29.7MB/s 
     |████████████████████████████████| 1.5MB 64.3MB/s 
     |████████████████████████████████| 460kB 57.3MB/s 
     |████████████████████████████████| 286kB 27.0MB/s 
     |████████████████████████████████| 5.4MB 43.6MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449910 sha256=e4dd7dc91b67405534f5e8068a5e77a93b6059ff39a9d600eb3b91349905f91b
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for clint: filename=clint-0.5.1-cp36-none-any.whl size=34448 sha256=d07c6eb50471ed68855bacd4f8fbdca15dd6fcd123aba73a5c737d679c3d3ba0
  Stored in directory: /root/.cache/pip/wheels/4f/e9/45/223565e5b1a4b09e12c6de6f8ba7c2c0e9127dec17cf830f83
  Created wheel for args: filename=args-0.1.0-cp36-none-any.whl size=3320 sha256=8a509e7ff7c40c83af6be4114589106ac43ba93f5e9cfde94a2595f9c1d0da48
  Stor

In [0]:
import pandas as pd
import os
import json
import nltk
# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize 
from nltk import pos_tag
import numpy as np
import pickle
import string
import random
import timeit

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 

import warnings
warnings.simplefilter('ignore')

In [0]:
# cd drive/My Drive/Information Retrieval/TF-IDF-DOCUMENT-RETRIEVAL-CHATBOT/


Data Preparation - Convert CSV to JSON

In [5]:
path = '/content/drive/My Drive/Information Retrieval/TF-IDF-DOCUMENT-RETRIEVAL-CHATBOT/'
convdata = pd.read_csv(path+'science-question-vietnames.csv')

#show header of the dataset
convdata.head()

,MESSAGE,RESPONSE,GOOGLE
0,Đại dương – Wikipedia tiếng Việt,Vì là thành phần chủ yếu của thủy quyển Trái Đ...,Đại dương lớn nhất thế giới
1,"Đại dương nào lớn nhất thế giới, diện tích hơn...",25 Tháng 4 2019 ... Có diện tích hơn 160 triệu...,Đại dương lớn nhất thế giới
2,Đại dương nào lớn nhất trên Trái Đất? - VnExpress,20 Tháng Ba 2019 ... Một đại dương chiếm tới 1...,Đại dương lớn nhất thế giới
3,Thái Bình Dương – Wikipedia tiếng Việt,"Thái Bình Dương là đại dương lớn nhất địa cầu,...",Đại dương lớn nhất thế giới
4,"Đại dương nào lớn nhất hành tinh, diện tích hơ...",28 Tháng Chín 2018 ... Đây là một trong những ...,Đại dương lớn nhất thế giới


In [6]:
#covert dataframes to json
convdata_json = json.loads(convdata.to_json(orient='records', force_ascii=False),encoding='utf8')
convdata_json[0:12]

[{'GOOGLE': 'Đại dương lớn nhất thế giới',
  'MESSAGE': 'Đại dương – Wikipedia tiếng Việt',
  'RESPONSE': 'Vì là thành phần chủ yếu của thủy quyển Trái Đất, đại dương thế giới không ... \r\nKhối chất lỏng bề mặt ngoài Trái Đất lớn và ổn định được xác nhận duy nhất là\xa0...'},
 {'GOOGLE': 'Đại dương lớn nhất thế giới',
  'MESSAGE': 'Đại dương nào lớn nhất thế giới, diện tích hơn 160 triệu km2? - Tư ...',
  'RESPONSE': '25 Tháng 4 2019 ... Có diện tích hơn 160 triệu km2, đây chính là đại dương lớn nhất trên thế giới \r\nhiện nay.'},
 {'GOOGLE': 'Đại dương lớn nhất thế giới',
  'MESSAGE': 'Đại dương nào lớn nhất trên Trái Đất? - VnExpress',
  'RESPONSE': '20 Tháng Ba 2019 ... Một đại dương chiếm tới 1/3 tổng diện tích bề mặt địa cầu và sở hữu điểm sâu \r\nnhất của lớp vỏ Trái Đất. - VnExpress. ... Tòa tháp đôi cao nhất thế giới nằm ở \r\nđâu?'},
 {'GOOGLE': 'Đại dương lớn nhất thế giới',
  'MESSAGE': 'Thái Bình Dương – Wikipedia tiếng Việt',
  'RESPONSE': 'Thái Bình Dương là đại dương lớ

In [0]:
#export as data as JSON
with open(path+'conversation_json.json', 'w', encoding='utf8') as outfile:
    json.dump(convdata_json, outfile, ensure_ascii=False)

                            

Function Defination

In [0]:
#greeting function
GREETING_INPUTS = ("hello", "hi", "chào", "halo","hey")
GREETING_RESPONSES = ["Chào bạn, bạn cần tư vấn gì?", "Hello"]
           
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [0]:
#Wordnet Lemmatization 
#for English
lemmer = nltk.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [0]:
# Remove punctuation
def RemovePunction(tokens):
    return[t for t in tokens if t not in string.punctuation]

In [9]:
# Create a stopword list from the standard list of stopwords available in nltk
#use for English
# stop_words = set(stopwords.words('english'))
print(path)
with open(path + "vietnamese-stopword.txt", "r") as f:
    new_stopwords = []
    for line in f.readlines():
        new_stopwords.append(line[:-1])

stop_words = new_stopwords[1:]
print (stop_words)

print(len(stop_words))
# stop_words = set(path + "vietnamese-stopword.txt")
# print((stop_words))

/content/drive/My Drive/Information Retrieval/TF-IDF-DOCUMENT-RETRIEVAL-CHATBOT/
['a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy', 'ba', 'ba_ba', 'ba_bản', 'ba_cùng', 'ba_họ', 'ba_ngày', 'ba_ngôi', 'ba_tăng', 'bao_giờ', 'bao_lâu', 'bao_nhiêu', 'bao_nả', 'bay_biến', 'biết', 'biết_bao', 'biết_bao_nhiêu', 'biết_chắc', 'biết_chừng_nào', 'biết_mình', 'biết_mấy', 'biết_thế', 'biết_trước', 'biết_việc', 'biết_đâu', 'biết_đâu_chừng', 'biết_đâu_đấy', 'biết_được', 'buổi', 'buổi_làm', 'buổi_mới', 'buổi_ngày', 'buổi_sớm', 'bà', 'bà_ấy', 'bài', 'bài_bác', 'bài_bỏ', 'bài_cái', 'bác', 'bán', 'bán_cấp', 'bán_dạ', 'bán_thế', 'bây_bẩy', 'bây_chừ', 'bây_giờ', 'bây_nhiêu', 'bèn', 'béng', 'bên', 'bên_bị', 'bên_có', 'bên_cạnh', 'bông', 'bước', 'bước_khỏi', 'bước_tới', 'bước_đi', 'bạn', 'bản', 'bản_bộ', 'bản_riêng', 'bản_thân', 'bản_ý', 'bất_chợt', 'bất_cứ', 'bất_giác', 'bất_kì', 'bất_kể', 'bất_kỳ', 'bất_luận', 'bất_ngờ', 'bất_nhược', 'bất_quá', 'bất_quá_chỉ', 'bất_thình_lình', 'bất_t

In [0]:
from underthesea import word_tokenize

def Talk_To_Bot(test_set_sentence):
    json_file_path = path + "conversation_json.json" 
    tfidf_vectorizer_pickle_path = path + "tfidf_vectorizer.pkl"
    tfidf_matrix_pickle_path = path + "tfidf_matrix_train.pkl"
    
    sentences = []
    
    # ---------------Tokenisation of user input -----------------------------#
    tokens = RemovePunction(word_tokenize(test_set_sentence, format="text").split(' '))
    # for English
    # pos_tokens = [word for word,pos in pos_tag(tokens, tagset='universal')]
    # word_tokens = LemTokens(pos_tokens)
    word_tokens = tokens
    # print("word_token",word_tokens)
    
    filtered_sentence = []
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)  
    
    filtered_sentence =" ".join(filtered_sentence).lower()
            
    test_set = (filtered_sentence, "")

    
    #For Tracing, comment to remove from print.
    print("""USER'S INPUT AFTER FILTERING: """,filtered_sentence)
    
    # -----------------------------------------------------------------------#
        
    try: 
        # ---------------Use Pre-Train Model------------------#
        f = open(tfidf_vectorizer_pickle_path, 'rb')
        tfidf_vectorizer = pickle.load(f)
        f.close()
        
        f = open(tfidf_matrix_pickle_path, 'rb')
        tfidf_matrix_train = pickle.load(f)
        # ---------------------------------------------------#
    except: 
        # ---------------To Train------------------#
        
        start = timeit.default_timer()
        
        with open(json_file_path) as sentences_file:
            reader = json.load(sentences_file)
            
            # ---------------Tokenisation of training input -----------------------------#    
            
            for row in reader:
                db_tokens = RemovePunction(word_tokenize(row['MESSAGE'], format="text").split(' '))
                # for English
                # pos_db_tokens = [word for word,pos in pos_tag(db_tokens, tagset='universal')]
                # db_word_tokens = LemTokens(pos_db_tokens)
                db_word_tokens = db_tokens
                
                db_filtered_sentence = [] 
                for dbw in db_word_tokens: 
                    if dbw not in stop_words: 
                        db_filtered_sentence.append(dbw)  
                
                db_filtered_sentence =" ".join(db_filtered_sentence).lower()
                
                #Debugging Checkpoint
                print('TRAINING INPUT: '+ db_filtered_sentence)
                
                sentences.append(db_filtered_sentence)
            # ---------------------------------------------------------------------------#
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix_train = tfidf_vectorizer.fit_transform(sentences)
        
        #train timing
        stop = timeit.default_timer()
        print ("Training Time : ")
        print (stop - start) 
    
        f = open(tfidf_vectorizer_pickle_path, 'wb')
        pickle.dump(tfidf_vectorizer, f) 
        f.close()
    
        f = open(tfidf_matrix_pickle_path, 'wb')
        pickle.dump(tfidf_matrix_train, f) 
        f.close 
        # ------------------------------------------#
        
    #use the learnt dimension space to run TF-IDF on the query
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)

    #then run cosine similarity between the 2 tf-idfs
    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)

    req_tfidf = cosine.max()
    if (req_tfidf == 0): #Threshold A
        
        not_understood = "Xin lỗi, tôi không hiểu!"
        
        return not_understood, not_understood, 0
        
    else:
        Threshold_B = 0.1
        print("Shape of cosine matrix: ",np.shape(cosine))

        max = cosine.max()
        print("MAX SCORE:", max)
        response_index = 0
        #if max score is lower than < Threshold_B > (we see can ask if need to rephrase.)
        if (max <= Threshold_B): #Threshold B
            
            not_understood = "Xin lỗi, tôi không hiểu!"
            
            return not_understood,not_understood, 0
        else:

                #if score is more than Threshold_B list the multi response and get a random reply
                if (max > Threshold_B): #Threshold C
                    
                    # load them to a list
                    flatcosine = cosine.flatten()
                    listsuit = flatcosine.argsort()[-10:][::-1]
                    response_index = listsuit
                    print("best_indexes: ", response_index)
                # else:
                #     # else we would simply return the highest score
                #     response_index = np.where(cosine == max)[0][0] + 2 

              
                results = []
                with open(json_file_path, "r") as sentences_file:
                    reader = json.load(sentences_file)
                    for i in response_index:
                        j = 0 
                        for row in reader:
                            if ((j == i) and (flatcosine[i] >= Threshold_B)): 
                                results.append(tuple((row["RESPONSE"], row["MESSAGE"], flatcosine[i])))
                                print("RESPONSE: ", row["RESPONSE"],"\nORIGINAL MESSAGE: ", row["MESSAGE"], "\nSCORE: ",flatcosine[i])
                                print("\n")
    
                                break
                            j += 1 
                return results
# Talk_To_Bot("How can I adopt a daughter in Singapore?")
Talk_To_Bot('Đại dương lớn nhất thế giới')
# Talk_To_Bot("ple")

In [0]:
flag=True
print("......................................................................................")
print('\x1b[1;37;40m' + 'SMART BOT'+'\x1b[0m'+': '+ 'Chào bạn, Tôi là SMART BOT.')
print('\x1b[1;37;40m' + 'SMART BOT'+'\x1b[0m'+': '+ 'Tôi biết khá nhiều về khoa học, hi vọng tôi có thể giúp đỡ bạn')
print('\x1b[1;37;40m' + 'SMART BOT'+'\x1b[0m'+': '+ 'Nếu muốn dừng cuộc hội thoại, hãy gõ "bye"!')
while(flag==True):
    print("......................................................................................")
    sentence = input('\x1b[0;30;47m' +"Tôi  "+'\x1b[0m'+":")
    print("......................................................................................")
    if(sentence.lower()!='bye'):
        if(greeting(sentence.lower())!=None):
            print('\x1b[1;37;40m' + 'SMART BOT'+'\x1b[0m'+': '+ greeting(sentence.lower()))
        else:
            for x in Talk_To_Bot(sentence):
               
                print("\n=====================================")
                print("\nThe best result is: ")
                print('\x1b[1;37;40m' + 'SMART BOT'+'\x1b[0m'+': '+x[0])
                print("\nSCORE: "+str(x[2]))
                print("\nORIGINAL_QUESTION:"+x[1])
                print("")
                break
    else:
        flag=False
print('\x1b[1;37;40m' + 'SMART BOT'+'\x1b[0m'+': '+"Chào bạn, hẹn gặp lại!") 

......................................................................................
SMART BOT: Chào bạn, Tôi là SMART BOT.
SMART BOT: Tôi biết khá nhiều về khoa học, hi vọng tôi có thể giúp đỡ bạn
SMART BOT: Nếu muốn dừng cuộc hội thoại, hãy gõ "bye"!
......................................................................................
Tôi  :halo
......................................................................................
SMART BOT: Hello
......................................................................................
Tôi  :núi cao
......................................................................................
USER'S INPUT AFTER FILTERING:  núi
Shape of cosine matrix:  (2, 138)
MAX SCORE: 0.8558008586920761
best_indexes:  [125 128 136 135 137 134   4  95  89  90]
RESPONSE:  Núi là dạng địa hình lồi, có sườn dốc và độ cao thường lớn hơn đồi, nằm trải dài 
... dãy Hymalaya với độ cao 8848 m tính từ mặt nước biển, là đỉnh núi cao nhất ... 
ORIGINAL MESSAGE:  Núi